# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import time
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
from scipy.io import loadmat
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image
from skimage.feature import hessian_matrix, hessian_matrix_eigvals
from scipy.ndimage.filters import convolve
from skimage import data, io, filters
import skimage
from skimage.morphology import convex_hull_image, erosion
from IPython import display
from scipy.ndimage import gaussian_filter
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from tensorflow.keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose,\
LeakyReLU, GaussianNoise, GlobalMaxPooling2D, ReLU, Input, Concatenate
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
from keras.datasets import mnist
import keras
from keras.models import Model
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PROCESS SUMMARY

### OVERVIEW

* Example Path Process
* Example Image Display
* Example MAT Load
* Example Drawing Marker
* Example Transformation Of Coordinates
* Example Coordinates Overwriting Image
* Example Density Display

#### EXAMPLE PATH

In [ ]:
Example_Image_Path = "../input/shanghaitech/ShanghaiTech/part_A/train_data/images/IMG_1.jpg"

In [ ]:
Example_Test_Path = "../input/shanghaitech/ShanghaiTech/part_A/test_data/images/IMG_1.jpg"

# we need this for example demonstration because cv2 processes overwrite images

#### EXAMPLE IMAGE

In [ ]:
Example_Reading_Image = cv2.cvtColor(cv2.imread(Example_Image_Path),cv2.COLOR_BGR2RGB)

In [ ]:
Example_Test_Image = cv2.cvtColor(cv2.imread(Example_Test_Path),cv2.COLOR_BGR2RGB)

# for example demonstration

In [ ]:
figure = plt.figure(figsize=(10,10))

plt.imshow(Example_Reading_Image)

In [ ]:
figure = plt.figure(figsize=(10,10))

plt.imshow(Example_Test_Image)

#### EXAMPLE ZEROS FOR PROTECTING MAIN IMAGE

In [ ]:
Example_Testing_Zeros = np.zeros((Example_Reading_Image.shape[0], Example_Reading_Image.shape[1]), dtype=np.float32)

In [ ]:
figure = plt.figure(figsize=(10,10))

plt.imshow(Example_Testing_Zeros)

#### EXAMPLE MAT PATH

In [ ]:
Example_Mat_Path = "../input/shanghaitech/ShanghaiTech/part_A/train_data/ground-truth/GT_IMG_1.mat"

In [ ]:
Example_Testing_Mat = "../input/shanghaitech/ShanghaiTech/part_A/test_data/ground-truth/GT_IMG_1.mat"

#### EXAMPLE LOAD MAT

In [ ]:
Example_Load_Mat = loadmat(Example_Mat_Path)

In [ ]:
print(type(Example_Load_Mat))

In [ ]:
print(Example_Load_Mat.items())

In [ ]:
print(Example_Load_Mat.keys())

# we need image info
# image_info is a keyword in the MAT dictionary.

In [ ]:
Example_Image_Info = Example_Load_Mat.get("image_info")

In [ ]:
print(type(Example_Image_Info))

In [ ]:
print(Example_Image_Info.shape)

In [ ]:
print(Example_Image_Info)

# we need coordinates

In [ ]:
Example_Coordinates = Example_Image_Info[0][0][0][0][0]

# coordinates

In [ ]:
print(Example_Coordinates)

In [ ]:
print(type(Example_Coordinates))

In [ ]:
print(Example_Coordinates.shape)

#### EXAMPLE DEMONSTRATION

In [ ]:
Example_Load_Mat_For_Testing = loadmat(Example_Testing_Mat)["image_info"]

In [ ]:
Example_Testing_Coordinates = Example_Load_Mat_For_Testing[0][0][0][0][0]

In [ ]:
print(Example_Testing_Coordinates[0:10])

In [ ]:
figure = plt.figure(figsize=(10,10))

for e_x_cor, e_y_cor in Example_Testing_Coordinates:
    e_x_cor = int(e_x_cor)
    e_y_cor = int(e_y_cor)
    cv2.drawMarker(Example_Test_Image, (e_x_cor, e_y_cor), (255, 0, 0),thickness=1)

plt.imshow(Example_Test_Image)
plt.title("testing")

#### EXAMPLE TRANSFORMATION OF COORDINATES

In [ ]:
for x_cor_example, y_cor_example in Example_Coordinates:
    x_cor_example = int(x_cor_example)
    y_cor_example = int(y_cor_example)
    Example_Testing_Zeros[y_cor_example, x_cor_example] = 1
    
figure = plt.figure(figsize=(10,10))
plt.imshow(Example_Testing_Zeros,cmap="gray")

#### EXAMPLE PEOPLE DENSITY

In [ ]:
Gaussian_Image_Example = gaussian_filter(Example_Testing_Zeros,sigma=5,truncate=5*5)

figure = plt.figure(figsize=(10,10))
plt.xlabel(Gaussian_Image_Example.shape)
plt.imshow(Gaussian_Image_Example,cmap="jet")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))

axis[0].imshow(Example_Reading_Image)
axis[1].imshow(Example_Testing_Zeros)
axis[2].imshow(Gaussian_Image_Example)

# DATA PROCESS

#### MAIN PATH

In [ ]:
Ground_Truth_Path_A = Path("../input/shanghaitech/ShanghaiTech/part_A/train_data/ground-truth")

In [ ]:
Images_Path_A = Path("../input/shanghaitech/ShanghaiTech/part_A/train_data/images")

In [ ]:
Ground_Truth_Path_B = Path("../input/shanghaitech/ShanghaiTech/part_B/train_data/ground-truth")

In [ ]:
Images_Path_B = Path("../input/shanghaitech/ShanghaiTech/part_B/train_data/images")

#### COORDINATES AND IMAGE PATH

In [ ]:
GT_List_A = list(Ground_Truth_Path_A.glob(r"*.mat"))
GT_List_B = list(Ground_Truth_Path_B.glob(r"*.mat"))

In [ ]:
Images_List_A = list(Images_Path_A.glob(r"*.jpg"))
Images_List_B = list(Images_Path_B.glob(r"*.jpg"))

In [ ]:
GT_List_A = sorted(GT_List_A)
GT_List_B = sorted(GT_List_B)
Images_List_A = sorted(Images_List_A)
Images_List_B = sorted(Images_List_B)

In [ ]:
print("LEN A GT: ",len(GT_List_A))
print("---"*5)
print("LEN A IMAGES: ",len(Images_List_A))
print("---"*20)
print("LEN B GT: ",len(GT_List_B))
print("---"*5)
print("LEN B IMAGES: ",len(Images_List_B))

In [ ]:
print(GT_List_A[0])
print(Images_List_A[0])

In [ ]:
print(GT_List_A[100])
print(Images_List_A[100])

#### TOTAL MAT AND IMAGES

In [ ]:
Total_Mat_Path = []
Total_Image_Path = []

for ma_path,mb_path,ia_path,ib_path in zip(GT_List_A,GT_List_B,Images_List_A,Images_List_B):
    Total_Mat_Path.append(ma_path)
    Total_Mat_Path.append(mb_path)
    Total_Image_Path.append(ia_path)
    Total_Image_Path.append(ib_path)

In [ ]:
print("MAIN MAT LEN: ",len(Total_Mat_Path))
print("---"*5)
print("MAIN IMAGE LEN: ",len(Total_Image_Path))

In [ ]:
print(Total_Mat_Path[0])
print(Total_Image_Path[0])

In [ ]:
print(Total_Mat_Path[100])
print(Total_Image_Path[100])

In [ ]:
Image_Path_Series = pd.Series(Total_Image_Path,name="IMAGE").astype(str)
Mat_Path_Series = pd.Series(Total_Mat_Path,name="MAT").astype(str)

In [ ]:
print("MAT LEN: ",len(Mat_Path_Series))
print("---"*5)
print("IMAGE LEN: ",len(Image_Path_Series))

In [ ]:
print(Image_Path_Series[44])

In [ ]:
print(Mat_Path_Series[44])

#### CREATING TRAIN DATA

In [ ]:
Image_Label_List = []
Mat_Label_List = []

total_added = []
total_not_added = []
time_count = 0

for x_images,x_mat in zip(Image_Path_Series.values,Mat_Path_Series.values):
    
    try:
        time_count += 1
        Reading_Image = cv2.cvtColor(cv2.imread(x_images),cv2.COLOR_BGR2RGB)

        Coordinates_x_info = loadmat(x_mat)["image_info"]
        Coordinates_x = Coordinates_x_info[0][0][0][0][0]

        Main_Zeros = np.zeros((Reading_Image.shape[0], Reading_Image.shape[1]), dtype=np.float32)
        for x_cor, y_cor in Coordinates_x:
            x_cor = int(x_cor)
            y_cor = int(y_cor)
            Main_Zeros[y_cor, x_cor] = 1

        Gaussian_Image_With_Coordinates = gaussian_filter(Main_Zeros,sigma=6,truncate=6*6)
        
        Resized_Image = cv2.resize(Reading_Image,(180,180))
        Resized_Gaussian_Image = cv2.resize(Gaussian_Image_With_Coordinates,(180,180))
        
        Image_Label_List.append(Resized_Image)
        Mat_Label_List.append(Resized_Gaussian_Image)
        total_added.append(str(time_count))
        
    except :
        total_not_added.append(str(time_count))
        pass

print(f"{len(total_not_added)} EPOCHS NOT ADDED")
print("---"*20)
print(f"{len(total_added)} EPOCHS ADDED")
        
        
    

#### CHECKING NEW DATA

In [ ]:
print("MAT LEN: ",len(Mat_Label_List))
print("IMAGE LEN: ",len(Image_Label_List))

In [ ]:
print("MAT IMAGE SHAPE: ",Mat_Label_List[0].shape)
print("ORIGINAL IMAGE SHAPE: ",Image_Label_List[0].shape)

In [ ]:
print("WHEN IT IS ARRAY MAT SHAPE: ",np.shape(np.array(Mat_Label_List)))
print("WHEN IT IS ARRAY IMAGE SHAPE: ",np.shape(np.array(Image_Label_List)))

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[0],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[0].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[0])
axis[1].set_xlabel(Image_Label_List[0].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[220],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[220].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[220])
axis[1].set_xlabel(Image_Label_List[220].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[22],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[22].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[22])
axis[1].set_xlabel(Image_Label_List[22].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[302],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[302].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[302])
axis[1].set_xlabel(Image_Label_List[302].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[552],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[552].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[552])
axis[1].set_xlabel(Image_Label_List[552].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[9],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[9].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[9])
axis[1].set_xlabel(Image_Label_List[9].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[276],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[276].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[276])
axis[1].set_xlabel(Image_Label_List[276].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(Mat_Label_List[16],cmap="jet")
axis[0].set_xlabel(Mat_Label_List[16].shape)
axis[0].set_title("MAT")
axis[1].imshow(Image_Label_List[16])
axis[1].set_xlabel(Image_Label_List[16].shape)
axis[1].set_title("ORIGINAL")

In [ ]:
figure,axis = plt.subplots(5,5,figsize=(10,10))

for indexing,operation in enumerate(axis.flat):
    
    Mat_Picking_Image = Mat_Label_List[indexing]
    operation.imshow(Mat_Picking_Image)
    
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(5,5,figsize=(10,10))

for indexing,operation in enumerate(axis.flat):
    
    Image_Picking_Image = Image_Label_List[indexing]
    operation.imshow(Image_Picking_Image)
    
plt.tight_layout()
plt.show()

#### TO ARRAY

In [ ]:
X_Train = np.array(Image_Label_List,dtype="float32")
X_Test = np.array(Mat_Label_List,dtype="float32")

In [ ]:
print("TRAIN SHAPE: ",X_Train.shape)
print("TEST SHAPE: ",X_Test.shape)

In [ ]:
X_Train = X_Train / 255.
X_Test = X_Test / 255.

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))

axis[0].imshow(X_Test[1],cmap="jet")
axis[0].set_xlabel(X_Test[1].shape)
axis[0].set_title("MAT")
axis[1].imshow(X_Train[1])
axis[1].set_xlabel(X_Train[1].shape)
axis[1].set_title("ORIGINAL")

# CONV2D - CONV2DTRANPOSE AUTO ENCODER PROCESS

#### CALLBACK

In [ ]:
Early_Stopper = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=3,mode="min")
Checkpoint_Model = tf.keras.callbacks.ModelCheckpoint(monitor="val_accuracy",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      filepath="./modelcheck")

#### PARAMETERS

In [ ]:
compile_loss = "mse"
compile_optimizer = Adam(lr=0.0000001)
output_class = 1

#### ENCODER

In [ ]:
Encoder_AE = Sequential()
Encoder_AE.add(Conv2D(32,(2,2),kernel_initializer = 'he_normal'))
Encoder_AE.add(BatchNormalization())
Encoder_AE.add(ReLU())
#
Encoder_AE.add(Conv2D(64,(2,2),kernel_initializer = 'he_normal'))
Encoder_AE.add(BatchNormalization())
Encoder_AE.add(ReLU())
#
Encoder_AE.add(Conv2D(128,(2,2),kernel_initializer = 'he_normal'))
Encoder_AE.add(BatchNormalization())
Encoder_AE.add(ReLU())
#
Encoder_AE.add(Conv2D(256,(2,2),kernel_initializer = 'he_normal'))
Encoder_AE.add(BatchNormalization())
Encoder_AE.add(ReLU())

#### DECODER

In [ ]:
Decoder_AE = Sequential()
Decoder_AE.add(Conv2DTranspose(128,(2,2)))
Decoder_AE.add(ReLU())
#
Decoder_AE.add(Conv2DTranspose(64,(2,2)))
Decoder_AE.add(ReLU())
#
Decoder_AE.add(Conv2DTranspose(32,(2,2)))
Decoder_AE.add(ReLU())
#
Decoder_AE.add(Conv2DTranspose(output_class,(2,2)))
Decoder_AE.add(ReLU())

#### AUTO ENCODER STRUCTURE

In [ ]:
Auto_Encoder = Sequential([Encoder_AE,Decoder_AE])

In [ ]:
Auto_Encoder.compile(loss=compile_loss,optimizer=compile_optimizer)

#### AUTO ENCODER TRAINING

In [ ]:
Model_AutoEncoder_Histoyr = Auto_Encoder.fit(X_Train,X_Test,epochs=10,callbacks=[Checkpoint_Model])

#### SAMPLE PREDICTION

In [ ]:
Prediction_Seen = Auto_Encoder.predict(X_Train[:5])

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))
axis[0].imshow(X_Train[4])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[4],cmap="jet")
axis[1].set_title("PREDICTION")
axis[2].imshow(Prediction_Seen[4],cmap="Spectral")
axis[2].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))
axis[0].imshow(X_Train[1])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[1],cmap="jet")
axis[1].set_title("PREDICTION")
axis[2].imshow(Prediction_Seen[1],cmap="hot")
axis[2].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))
axis[0].imshow(X_Train[2])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[2],cmap="jet")
axis[1].set_title("PREDICTION")
axis[2].imshow(Prediction_Seen[2],cmap="cubehelix")
axis[2].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,3,figsize=(10,10))
axis[0].imshow(X_Train[3])
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Seen[3],cmap="jet")
axis[1].set_title("PREDICTION")
axis[2].imshow(Prediction_Seen[3],cmap="nipy_spectral")
axis[2].set_title("PREDICTION")

#### MEAN CHECKING

In [ ]:
figure,axis = plt.subplots(1,4,figsize=(10,10))
axis[0].imshow(Prediction_Seen[0],cmap="jet")
axis[0].set_title(np.mean(Prediction_Seen[0]))
axis[1].imshow(Prediction_Seen[1],cmap="jet")
axis[1].set_title(np.mean(Prediction_Seen[1]))
axis[2].imshow(Prediction_Seen[2],cmap="jet")
axis[2].set_title(np.mean(Prediction_Seen[2]))
axis[3].imshow(Prediction_Seen[3],cmap="jet")
axis[3].set_title(np.mean(Prediction_Seen[3]))

In [ ]:
Mean_One = np.mean(Prediction_Seen[0])
Mean_Two = np.mean(Prediction_Seen[1])
Mean_Three = np.mean(Prediction_Seen[2])
Mean_Four = np.mean(Prediction_Seen[3])

print(Mean_One)
print(Mean_Two)
print(Mean_Three)
print(Mean_Four)

#### SPECIAL PREDICTION

In [ ]:
Non_Seen_Image_One = "../input/shanghaitech/ShanghaiTech/part_A/test_data/images/IMG_110.jpg"
Non_Seen_Image_Two = "../input/shanghaitech/ShanghaiTech/part_A/test_data/images/IMG_123.jpg"
Non_Seen_Image_Three = "../input/shanghaitech/ShanghaiTech/part_A/test_data/images/IMG_100.jpg"

##### ONE

In [ ]:
Reading_Sample = cv2.cvtColor(cv2.imread(Non_Seen_Image_One),cv2.COLOR_BGR2RGB)
Resized_Sample = cv2.resize(Reading_Sample,(180,180))
Prediction_Resized_Sample = Resized_Sample.reshape(-1,180,180,3)

In [ ]:
Prediction_Non_Seen = Auto_Encoder.predict(Prediction_Resized_Sample)
Prediction_Non_Seen = Prediction_Non_Seen.reshape(180,180)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Resized_Sample)
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen,cmap="nipy_spectral")
axis[1].set_title("PREDICTION")

##### TWO

In [ ]:
Reading_Sample = cv2.cvtColor(cv2.imread(Non_Seen_Image_Two),cv2.COLOR_BGR2RGB)
Resized_Sample = cv2.resize(Reading_Sample,(180,180))
Prediction_Resized_Sample = Resized_Sample.reshape(-1,180,180,3)

Prediction_Non_Seen = Auto_Encoder.predict(Prediction_Resized_Sample)
Prediction_Non_Seen = Prediction_Non_Seen.reshape(180,180)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Resized_Sample)
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen,cmap="nipy_spectral")
axis[1].set_title("PREDICTION")

##### THREE

In [ ]:
Reading_Sample = cv2.cvtColor(cv2.imread(Non_Seen_Image_Three),cv2.COLOR_BGR2RGB)
Resized_Sample = cv2.resize(Reading_Sample,(180,180))
Prediction_Resized_Sample = Resized_Sample.reshape(-1,180,180,3)

Prediction_Non_Seen = Auto_Encoder.predict(Prediction_Resized_Sample)
Prediction_Non_Seen = Prediction_Non_Seen.reshape(180,180)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Resized_Sample)
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen,cmap="nipy_spectral")
axis[1].set_title("PREDICTION")

# SPECTRUM AUTO ENCODER PROCESS

#### STRUCTURE

In [ ]:
Encoder = Sequential()
#
Encoder.add(Conv2D(32,(2,2),kernel_initializer = 'he_normal'))
Encoder.add(BatchNormalization())
Encoder.add(LeakyReLU())
#
Encoder.add(Conv2D(64,(2,2),kernel_initializer = 'he_normal'))
Encoder.add(BatchNormalization())
Encoder.add(LeakyReLU())


Decoder = Sequential()
#
Decoder.add(Conv2DTranspose(32,(2,2)))
Decoder.add(LeakyReLU())
#
Decoder.add(Conv2DTranspose(output_class,(2,2)))
Decoder.add(LeakyReLU())

In [ ]:
ANN_AutoEncoder = Sequential([Encoder,Decoder])

In [ ]:
ANN_AutoEncoder.compile(loss="mse",optimizer=RMSprop(lr=0.000001))

#### TRAINING

In [ ]:
Hıstory_Complex = ANN_AutoEncoder.fit(X_Train,X_Test,epochs=10)

#### SAMPLE PREDICTION

In [ ]:
ANN_Prediction_Images = ANN_AutoEncoder.predict(X_Train[:5])

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(X_Train[1])
axis[0].set_title("ORIGINAL")
axis[1].imshow(ANN_Prediction_Images[1],cmap="nipy_spectral")
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(X_Train[2])
axis[0].set_title("ORIGINAL")
axis[1].imshow(ANN_Prediction_Images[2],cmap="jet")
axis[1].set_title("PREDICTION")

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(X_Train[3])
axis[0].set_title("ORIGINAL")
axis[1].imshow(ANN_Prediction_Images[3],cmap="hot")
axis[1].set_title("PREDICTION")

#### SPECIAL PREDICTION

##### ONE

In [ ]:
Non_Seen_Image = "../input/shanghaitech/ShanghaiTech/part_A/test_data/images/IMG_151.jpg"

Reading_Sample = cv2.cvtColor(cv2.imread(Non_Seen_Image),cv2.COLOR_BGR2RGB)
Resized_Sample = cv2.resize(Reading_Sample,(180,180))
Prediction_Resized_Sample = Resized_Sample.reshape(-1,180,180,3)

Prediction_Non_Seen = ANN_AutoEncoder.predict(Prediction_Resized_Sample)
Prediction_Non_Seen = Prediction_Non_Seen.reshape(180,180)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Resized_Sample)
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen,cmap="hot")
axis[1].set_title("PREDICTION")

##### TWO

In [ ]:
Non_Seen_Image = "../input/shanghaitech/ShanghaiTech/part_A/test_data/images/IMG_149.jpg"

Reading_Sample = cv2.cvtColor(cv2.imread(Non_Seen_Image),cv2.COLOR_BGR2RGB)
Resized_Sample = cv2.resize(Reading_Sample,(180,180))
Prediction_Resized_Sample = Resized_Sample.reshape(-1,180,180,3)

Prediction_Non_Seen = ANN_AutoEncoder.predict(Prediction_Resized_Sample)
Prediction_Non_Seen = Prediction_Non_Seen.reshape(180,180)

In [ ]:
figure,axis = plt.subplots(1,2,figsize=(10,10))
axis[0].imshow(Resized_Sample)
axis[0].set_title("ORIGINAL")
axis[1].imshow(Prediction_Non_Seen,cmap="hot")
axis[1].set_title("PREDICTION")